# Mathematics for AI - Coursework Task 2

The second task is about classifying handwritten digits. We will use the MNIST dataset for training and testing. The point of this task is to develop a multi-layer neural network for classificationusing mostly Numpy:

• Implement sigmoid and relu layers (with forward and backward pass)

• Implement a softmax output layer

• Implement a fully parameterizable neural network (number and types of layers, number of units)

• Implement an optimizer(e.g. SGD or Adam)and a stopping criterionof your choosing
• Train your Neural Network using backpropagation. Evaluate different neural network architectures andcompare your different results. You can also compare withthe results presented inhttp://yann.lecun.com/exdb/mnist/